<a href="https://colab.research.google.com/github/GeeveGeorge/Real-ESRGAN-Video-Batch-Process/blob/main/Real_ESRGAN_Batch_Process_Videos_Notebook_by_Geeve_George.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Important Notes

* If you are using the free Colab tier , it works best for short length videos
* Just follow the instructions in the notebook and you should be fine!

*Colab Notebook prepared by Geeve George*

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 561, done.
remote: Total 561 (delta 0), reused 0 (delta 0), pack-reused 561
Receiving objects: 100% (561/561), 4.64 MiB | 42.77 MiB/s, done.
Resolving deltas: 100% (288/288), done.
/content/Real-ESRGAN
     |████████████████████████████████| 161 kB 28.4 MB/s 
     |████████████████████████████████| 5.8 MB 59.0 MB/s 
     |████████████████████████████████| 190 kB 72.8 MB/s 
  Created wheel for basicsr: filename=basicsr-1.3.4.9-py3-none-any.whl size=194440 sha256=f01ab7a01d65c4ffb932935266e27e2403aee9fdcaa7ff4147fe38d80abc7ad3
  Stored in directory: /root/.cache/pip/wheels/92/52/70/341916b8c102f1e4416695e601d4aa71ba34d1080f7aa47cf5
Successfully built basicsr
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████████████████████████████| 177 kB 54.5 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=a81fca70420220ee2d95e802f08ae4d959b7553d9b9b51278f9b6a687933d2ea
  Store

# 2. This section is for creating some important folders (IMPORTANT : Please Read)



*   First Run this cell

*   Once you've run the cell , you can see that a folder named "videos" will be created inside the Real-ESRGAN folder


*   Just drag and drop any number of videos from your computer and place them inside the "videos" folder. (videos you want to upscale)







In [ ]:
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'
video_folder = 'videos'
video_result_folder = 'results_videos'
video_mp4_result_folder = 'results_mp4_videos'

if os.path.isdir(upload_folder):
  print(upload_folder+" exists")
else :
  os.mkdir(upload_folder)

if os.path.isdir(video_folder):
  print(video_folder+" exists")
else :
  os.mkdir(video_folder)

if os.path.isdir(video_result_folder):
  print(video_result_folder+" exists")
else :
  os.mkdir(video_result_folder)

if os.path.isdir(video_mp4_result_folder):
  print(video_mp4_result_folder+" exists")
else :
  os.mkdir(video_mp4_result_folder)

if os.path.isdir(result_folder):
  print(result_folder+" exists")
  
else :
  os.mkdir(result_folder)  




# 3. Inference (Please READ)



*   Then run this cell , it will take a good amount of time depending on the length of your video and how many videos you want to upscale

*   Once the upscaling process is complete , you can find your upscaled video results , under the "results_mp4_videos" folder.



*   By default the video is upscaled to a 25fps format,  in case you want to change that , it can be changed within the cell below , 
change the "fps" variable. (Can be found  towards the end part of the code section.)








In [ ]:
import cv2
import numpy as np
import glob
from os.path import isfile, join
import subprocess
from IPython.display import clear_output  

# assign directory
directory = '/content/Real-ESRGAN/videos' #PATH_WITH_INPUT_VIDEOS
zee = 0

#deletes frames from previous video
for f in os.listdir(upload_folder): 
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir(result_folder): 
    os.remove(os.path.join(result_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder): 
    os.remove(os.path.join(video_result_folder, f))
     
#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder): 
    os.remove(os.path.join(video_mp4_result_folder, f))


def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
    size2 = (0,0)

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        size2 = size
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size2)
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
    
        
for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      
      
      print("PROCESSING :"+str(f)+"\n")
      # Read the video from specified path
    
      #video to frames
      cam = cv2.VideoCapture(str(f))

      try:

          # PATH TO STORE VIDEO FRAMES
          if not os.path.exists('/content/Real-ESRGAN/upload'):
              os.makedirs('/content/Real-ESRGAN/upload')

      # if not created then raise error
      except OSError:
          print ('Error: Creating directory of data')

      # frame
      currentframe = 0

      #clear all folders


      #deletes upscaled frames from previous video
      #for f in os.listdir(result_folder):
      #  os.remove(os.path.join(result_folder, f))      
      



      while(True):

          # reading from frame
          ret,frame = cam.read()

          if ret:
              # if video is still left continue creating images
              name = '/content/Real-ESRGAN/upload/frame' + str(currentframe) + '.jpg'

              # writing the extracted images
              cv2.imwrite(name, frame)


                # increasing counter so that it will
                # show how many frames are created
              currentframe += 1
          else:
              #deletes all the videos you uploaded for upscaling
              #for f in os.listdir(video_folder):
              #  os.remove(os.path.join(video_folder, f)) 

              break
      
        # Release all space and windows once done
      cam.release()
      cv2.destroyAllWindows()
      
      #apply super-resolution on all frames of a video
      #scale factor is by 3.5x  

      !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale 2 --half --face_enhance --tile 128

      #after upscaling just delete the source frames
      for f in os.listdir(upload_folder): 
          os.remove(os.path.join(upload_folder, f))     

      #rename all frames in "results" to remove the 'out' substring from the processing results
      paths = (os.path.join(root, filename)
              for root, _, filenames in os.walk('/content/Real-ESRGAN/results')
              for filename in filenames)
      for path in paths:
          newname = path.replace('_out', '')
          if newname != path:
              os.rename(path, newname)      
      
      #convert super res frames to .avi  
      pathIn = '/content/Real-ESRGAN/results/'

      zee = zee+1
      fName = "video"+str(zee)  
      filenameVid = f"{fName}.avi"
        
      pathOut = "/content/Real-ESRGAN/results_videos/"+filenameVid

      fps = 25.0 #change this to FPS of your source video

      convert_frames_to_video(pathIn, pathOut, fps)      

      #after processing frames converted to .avi video , delete upscaled frames from previous video
      for f in os.listdir(result_folder): 
          os.remove(os.path.join(result_folder, f))

      #convert .avi to .mp4
      src = '/content/Real-ESRGAN/results_videos/'
      dst = '/content/Real-ESRGAN/results_mp4_videos/'

      for root, dirs, filenames in os.walk(src, topdown=False):
          #print(filenames)
          for filename in filenames:
              print('[INFO] 1',filename)
              try:
                  _format = ''
                  if ".flv" in filename.lower():
                      _format=".flv"
                  if ".mp4" in filename.lower():
                      _format=".mp4"
                  if ".avi" in filename.lower():
                      _format=".avi"
                  if ".mov" in filename.lower():
                      _format=".mov"

                  inputfile = os.path.join(root, filename)
                  print('[INFO] 1',inputfile)
                  outputfile = os.path.join(dst, filename.lower().replace(_format, ".mp4"))
                  subprocess.call(['ffmpeg', '-i', inputfile, outputfile])  
              except:
                  print("An exception occurred")        
  
      clear_output(wait=True)

      #clearing previous .avi files
      for f in os.listdir(video_result_folder): 
          os.remove(os.path.join(video_result_folder, f))

      #deletes frames from previous video
      #for f in os.listdir(upload_folder):
      #  os.remove(os.path.join(upload_folder, f))

        

      # if it is out of memory, try to use the `--tile` option
# We upsample the image with the scale factor X3.5

# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore. 

PROCESSING :/content/Real-ESRGAN/videos/death-note-light-yagami.mp4

Testing 0 frame0
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
Testing 1 frame1
	Tile 1/15
	Tile 2/15
	Tile 3/15
	Tile 4/15
	Tile 5/15
	Tile 6/15
	Tile 7/15
	Tile 8/15
	Tile 9/15
	Tile 10/15
	Tile 11/15
	Tile 12/15
	Tile 13/15
	Tile 14/15
	Tile 15/15
Testing 2 frame10
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "
	Tile 1/15
	Tile 2/15
	Tile 

# 4. Download Results



*   All your upscaled .mp4 files will be inside the results_mp4_videos folder (inside Real-ESRGAN folder)


*   You can right click on the needed file and download it from there.






# 5. Run this cell after the batch of videos have been upscaled 
# (WARNING : Deletes processed data)



*   IMPORTANT : Read the comments inside the code section , because this DELETES previous frames and videos


In [ ]:


#deletes frames from previous video
for f in os.listdir(upload_folder): 
    os.remove(os.path.join(upload_folder, f))

#deletes upscaled frames from previous video
for f in os.listdir(result_folder): 
    os.remove(os.path.join(result_folder, f))

#deletes all the videos you uploaded for upscaling
for f in os.listdir(video_folder): 
    os.remove(os.path.join(video_folder, f))

#clearing previous .avi files
for f in os.listdir(video_result_folder): 
    os.remove(os.path.join(video_result_folder, f))
     
#clearing .mp4 result files
for f in os.listdir(video_mp4_result_folder): 
    os.remove(os.path.join(video_mp4_result_folder, f))